### Deployment experimental notebook

model 1

latitude,property_type,room_type,accommodates,bedrooms,beds,bbq,tv,white_goods,bathrooms_type,bathrooms_number,bus_radius,within_radius,average_nearest_price,longitude

[49.2836,5,0,2,1.0,1.0,0.0,1.0,1.0,2,1.0,0,1,776000.0,-123.13254]

model 2

latitude,property_type,room_type,accommodates,bedrooms,beds,bbq,tv,white_goods,bathrooms_type,bathrooms_number,bus_radius,within_radius,average_nearest_price,longitude,nearest_airbnb_price_8

[49.22696,10,2,1,1.0,1.0,0.0,0.0,1.0,1,1.0,1,0,1989820.0,-123.16006,67.6]

Model 1

host_total_listings_count,high_end_electronics,bed_linen,coffee_machine,cooking_basics,child_friendly,parking,outdoor_space,internet,bus_radius,within_radius,average_nearest_price,longitude,latitude,price

2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1,1,723200.0,-123.0694956,49.2796307,119.0

Model 2

host_total_listings_count,high_end_electronics,bed_linen,coffee_machine,cooking_basics,child_friendly,parking,outdoor_space,internet,bus_radius,within_radius,average_nearest_price,longitude,latitude,price,nearest_airbnb_review_8

6,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1,0,963600.0,-123.10189,49.24008,61.0,4.454

In [1]:
# import pickle
# # save the model to disk
# filename = 'rf01.sav'
# pickle.dump(rf, open(filename, 'wb'))

# import pickle
# # save the model to disk
# filename = 'rf02.sav'
# pickle.dump(rf_2, open(filename, 'wb'))

In [2]:
import pandas as pd
from haversine import haversine, Unit
import rtree.index
import numpy as np

df_train_x = pd.read_csv('preprocessed_X_train.csv', sep=',')
df_train_y = pd.read_csv('preprocessed_y_train.csv', sep=',')

df_train = pd.concat([df_train_x, df_train_y], axis=1)


In [11]:
def feature_extrapolation_warning(array):
    if len(array) != 4:
        return
    var = ['accommodates', 'bedrooms', 'beds', 'bathrooms_number']
    inputs = {k: v for k, v in zip(var, array)}
    quantile = {k: v for k, v in zip(var, array)}
    # print(inputs)
    for i in var:
        quantile[i] = len(df_train[df_train[i] < inputs[i]]) / 3685
    
    warning_message = 'Extrapolation Warning: \nBy comparing with other Airbnb Vancouver listings, the listing you input may have the following aspects that may be a bit unusual. \n'
    warning_flag = False
    for i in var:
        if quantile[i] > 0.8 or inputs[i] <= 0:
            warning_flag = True
            warning_message += str('Your ' + i +  ' has a quantile of ' + str(quantile[i]*100) + '%\n')
    if not warning_flag:
        return False, ''
    else:
        return True, warning_message

In [13]:
response,a = feature_extrapolation_warning([1,1,0,1])
print(response,a)


True Extrapolation Warning: 
By comparing with other Airbnb Vancouver listings, the listing you input may have the following aspects that may be a bit unusual. 
Your beds has a quantile of 0.0%



In [6]:
def price_prediction_warning(price):       
    price_q = len(df_train[df_train['price'] < price]) / 3685
    
    if price >= 395 or price < 59.4:
        warning_message = 'Prediction Confidence Warning: \nBy comparing with existing Airbnb Vancouver listings price, this prediction may not be supported by sufficient available data. \n'
        # warning_message += str('This prediciton has a quantile of ' + str(price_q*100) + '%')
        return [True, warning_message]
    else:
        return [False, ""]

In [7]:
response = price_prediction_warning(350)
print(response[0])
print('--------')
print(response[1])

False
--------
No Extrapolation Warning
